In [1]:
import scanpy as sc
import pandas as pd
import numpy as np

In [6]:
!pip install scanpy==1.9

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement scanpy==1.9 (from versions: 0.2.1, 0.2.3, 0.2.3.4, 0.2.3.5, 0.2.4, 0.2.5, 0.2.6, 0.2.7, 0.2.8, 0.2.9, 0.2.9.1, 0.3, 0.3.1, 0.3.2, 0.4, 0.4.1, 0.4.2, 0.4.3, 0.4.4, 1.0, 1.0.1, 1.0.1.post1, 1.0.2, 1.0.3, 1.0.3.post1, 1.0.4, 1.1a1, 1.1a2, 1.1, 1.2.0, 1.2.1, 1.2.2, 1.3, 1.3.1, 1.3.2, 1.3.3, 1.3.4, 1.3.5, 1.3.6, 1.3.7, 1.3.8, 1.4, 1.4.1, 1.4.2, 1.4.3, 1.4.4, 1.4.4.post1, 1.4.5, 1.4.5.post1, 1.4.5.post2, 1.4.5.post3, 1.4.5.1, 1.4.6, 1.5.0a1, 1.5.0, 1.5.1, 1.6.0, 1.6.1, 1.7.0rc1, 1.7.0, 1.7.1, 1.7.2)
ERROR: No matching distribution found for scanpy==1.9


In [8]:
## scdata
scpath = '/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/data/scrnaseq/scVI_Atherosclerosis_Atlas_V2.h5ad'
sc_dat = sc.read_h5ad(scpath)


In [ ]:
#sc_dat.obs['Disease'].unique()
# get disease data
disease_scdat = sc_dat[sc_dat.obs['Disease'] == 'Healthy']
# disease_scdat = sc_dat[sc_dat.obs['Disease'] != 'Healthy']
# normalize
#disease_scdat = disease_scdat.raw.to_adata()
disease_scdat.var_names_make_unique() 
sc.pp.normalize_total(disease_scdat, target_sum=1e4)

In [3]:
disease_scdat.obs['Disease'].unique()


['Proximal plaques', 'type VII calcified plaques', 'atherosclerotic plaques', 'atherosclerotic lesions']
Categories (4, object): ['Proximal plaques', 'atherosclerotic lesions', 'atherosclerotic plaques', 'type VII calcified plaques']

In [ ]:
## stdata
stpath = r'D:\study\thesis\project\data\ST_plaque_all_donotshare_wo_permission.h5ad'
st_dat = sc.read_h5ad(stpath)
#st_dat.obs['Disease'].unique()
# get disease data
disease_scdat = st_dat[st_dat.obs['condition'] == 'Plaque',]
# normalize
#disease_scdat = disease_scdat.raw.to_adata()
disease_scdat.var_names_make_unique() 
sc.pp.normalize_total(disease_scdat, target_sum=1e4)

In [3]:
threshold = 1

In [4]:
# create co-exp matrix
def coexp_list(data, goi):
        res_array = data.multiply(data[goi], axis=0)
        coexp_count = np.count_nonzero(res_array, axis=0)
        coexp_frac = coexp_count/len(data)
        return coexp_frac
def coexp_matrix(adata, threshold):
    exp_dat = pd.DataFrame(adata.X.toarray(),columns=adata.var_names)
    exp_dat[exp_dat < threshold] = 0
    # Create an empty array with a specified shape (e.g., 0 rows, len(exp_dat.columns)) columns)
    matrix = np.empty((0, len(exp_dat.columns)))
    for gene in exp_dat.columns:
        temp_array = coexp_list(exp_dat, gene)
        # Append the new row to the empty array
        matrix = np.append(matrix, [temp_array], axis=0)
    return matrix

In [ ]:
threshold = 1
sc_all_cell_coxp_matrix = coexp_matrix(disease_scdat,threshold)
st_all_cell_coxp_matrix = coexp_matrix(disease_scdat,threshold)
## if you want to create matrix per cell type, modify the adata input for coexp_matrix
# storage = []
# celltypes = disease_scdat.obs['Main_Cell_Types'].tolist()
# for celltype in celltypes:
#     cell_adata = disease_scdat[disease_scdat.obs['Main_Cell_Types'] == celltype]
#     storage.append(coexp_matrix(cell_adata,threshold))